In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import year, weekofyear, count, expr, avg, sum as _sum
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt

In [0]:
spark = SparkSession.builder.appName("ModelTraining").getOrCreate()


In [0]:
# --- Cargar datos y agregar por semana ---
df_con_clima = spark.table("workspace.default.complete_dataset_con_clima")

df_weekly = (
    df_con_clima.withColumn("year", year("fecha_notificacion"))
      .withColumn("week", weekofyear("fecha_notificacion"))
      .groupBy("year", "week")
      .agg(
          count("*").alias("y"),
          avg("temperatura_media").alias("temperatura_media"),
          _sum("precipitacion").alias("precipitacion_total")
      )
      .withColumn("Semana", expr("date_add(concat(year, '-01-01'), (week - 1) * 7)"))
      .orderBy("Semana")
).toPandas()

In [0]:
# --- Ingeniería de Características ---
pdf = df_weekly.copy()
pdf["Semana"] = pd.to_datetime(pdf["Semana"])
pdf['temperatura_media'] = pdf['temperatura_media'].ffill().bfill()
pdf['precipitacion_total'] = pdf['precipitacion_total'].ffill().bfill()

pdf["week_of_year"] = pdf["Semana"].dt.isocalendar().week.astype(float)
pdf["sin_week"] = np.sin(2 * np.pi * pdf["week_of_year"] / 52)
pdf["cos_week"] = np.cos(2 * np.pi * pdf["week_of_year"] / 52)
pdf['prcp_lag_4'] = pdf['precipitacion_total'].shift(4)
pdf['y_lag_1'] = pdf['y'].shift(1)
pdf = pdf.dropna().reset_index(drop=True)

In [0]:
# --- Entrenar el Modelo ---
features = ["sin_week", "cos_week", "prcp_lag_4", "y_lag_1"]
X = sm.add_constant(pdf[features])
y = pdf["y"]
model = sm.OLS(y, X).fit()
features_with_const = X.columns.tolist()

print("--- RESUMEN DEL MODELO FINAL ---")
print(model.summary())

In [0]:
# --- Predicción Futura Recursiva ---
last_known_data = pdf.iloc[-1].copy()
predictions = []

for i in range(12):
    future_date = last_known_data["Semana"] + pd.Timedelta(weeks=1)
    future_week_of_year = future_date.isocalendar().week
    
    # de hace 4 semanas desde la fecha futura.
    prcp_source_date = future_date - pd.DateOffset(weeks=4)
    
    # Como no conocemos ese valor, lo estimamos usando la misma semana del año anterior.
    prcp_source_date_last_year = prcp_source_date - pd.DateOffset(years=1)
    
    # Encontramos el índice en nuestros datos históricos que está más cerca de esa fecha.
    closest_historical_index = np.argmin(abs(pdf['Semana'] - prcp_source_date_last_year))
    
    # Usamos el valor de precipitación de esa semana histórica como nuestra estimación.
    future_prcp_lag_4 = pdf.loc[closest_historical_index, 'precipitacion_total']
    
    future_y_lag_1 = last_known_data["y"]

    X_to_predict = pd.DataFrame([{'const': 1.0,
        'sin_week': np.sin(2 * np.pi * future_week_of_year / 52),
        'cos_week': np.cos(2 * np.pi * future_week_of_year / 52),
        'prcp_lag_4': future_prcp_lag_4,
        'y_lag_1': future_y_lag_1
    }])
    
    predicted_y = max(0, model.predict(X_to_predict[features_with_const])[0])
    predictions.append({'Semana': future_date, 'Casos Previstos': predicted_y})
    
    last_known_data['Semana'] = future_date
    last_known_data['y'] = predicted_y

future_pdf = pd.DataFrame(predictions)

In [0]:
# --- Visualización de Resultados ---
print("\n--- Visualizando los resultados del modelo ---")
plt.figure(figsize=(18, 8))
plt.plot(pdf['Semana'], pdf['y'], label='Casos Reales', color='blue', linewidth=2)
plt.plot(pdf['Semana'], model.fittedvalues, label='Ajuste del Modelo (Histórico)', color='red', linestyle='--')
plt.plot(future_pdf['Semana'], future_pdf['Casos Previstos'], label='Predicción Futura (12 semanas)', color='green', marker='o')
plt.title('Casos Semanales de Chikungunya: Reales vs. Predichos', fontsize=16)
plt.xlabel('Semana', fontsize=12)
plt.ylabel('Número de Casos Semanales', fontsize=12)
plt.legend(fontsize=10)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

print("\n--- PREDICCIONES PARA LAS PRÓXIMAS 12 SEMANAS ---")
display(future_pdf)
